<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></div>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (15,8)
from sklearn.model_selection import train_test_split
!pip install phik -q
import phik
from phik.report import plot_correlation_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from phik.report import plot_correlation_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
!pip install --upgrade scikit-learn 1.3.1
!pip install --upgrade pip
!pip install shap -q
import warnings
warnings.filterwarnings("ignore")

In [ ]:
taxi = pd.read_csv('/datasets/taxi.csv', index_col = [0], parse_dates = [0])
taxi.sort_index(inplace=True)

In [ ]:
taxi.head()

In [ ]:
taxi.info()

## Анализ

In [ ]:
taxi = taxi['2018-03-01':'2018-08-31'].resample('1H').sum()

In [ ]:
taxi.reset_index().head()

In [ ]:
taxi.plot()
plt.xlabel('период по месяцам')
plt.ylabel('количество заказов')
plt.title('Распределение заказов по месяцам')
plt.show();


In [ ]:
taxi.head()

In [ ]:
taxi.tail()

*Посмотрим на стационарность временного ряда*

In [ ]:
taxi['mean'] = taxi['num_orders'].shift().rolling(7).mean()
taxi['std'] = taxi['num_orders'].shift().rolling(7).std()

In [ ]:
taxi.plot()
plt.title('Распределение заказов по месяцам')
plt.xlabel('период по месяцам')
plt.ylabel('количество заказов')
plt.show();

*Сглаженный ряд скользящим средним увеличивается с течением времени. Также увеличивается cтандартное отклонение с течением времени. Это значит, что ряд нестационарный, его спрогнозировать сложнее.* 

*Ресэмплировали по часу указанный промежуток времени*

In [ ]:
taxi['dayofweek'] = taxi.index.dayofweek
taxi['hour'] = taxi.index.hour

In [ ]:
taxi_orders = taxi.groupby('dayofweek')['num_orders'].agg('mean')
taxi_orders.plot()
plt.xlabel('Дни недели')
plt.ylabel('Количество заказов')
plt.title('Распределение заказов по дням недели')
plt.show();

*Миниальное среднее количество заказов во вторник, около 75 заказов. Макимальное в пятницу, свыше 90 заказов.*

*Посмотрим по часам*

*Посмотрим статистику заказов с помощью метода describe*

In [ ]:
taxi_orders_hour = taxi.groupby('hour')['num_orders'].agg('mean')
taxi_orders_hour.plot()
plt.xlabel('Период по часам')
plt.ylabel('Количество заказов')
plt.title('Распределение заказов по часам внутри дня')
plt.show();


In [ ]:
taxi_orders_hour.describe()

*Миниальное количество заказов = 25 в 6 утра. Максимальное в 12 ночи, около 144.*

*Посмотрим на тренд и сезонность(в нашем случае как заказы распределются внутри дня*

*Чтобы посмотреть изменения внутри дня, возьмем данные за неделю*

In [ ]:
taxi_week = taxi['2018-08-25':'2018-08-31']
plt.xlabel('Период по дням')
plt.ylabel('Количество заказов')
plt.title('Распределение заказов по дням')
taxi_week.plot();

*Если посмотреть на часовой график 'hour', видно, что изменение заказов циклично(сезонно). Минимальное количество заказов утром с дальнейшим увелиением к вечеру и до поздней ночи, далее ближе к утру количество заказов падает. И так повторяется всю неделю ежедневно. Причиной такой сезонности, а именно увеличения заказов такси в аэропорт вечером и ночью считается, что большинство людей летят на самолете куда-то как правило после рабочего дня когда больше свободного времени либо заказывают такси в аэропорт, например чтобы встретить родственнико или друзей. Еще можно учесть фактор времен года. В нашем случае мы взяли период за неделю августа, когда люди в отпусках и летят отдыхать как правило во второй половине дня, поэтому и увеличивается количество заказов такси во второй половине дня.*

*Вывод - Загрузили данные, изучили общую информацию и ресэмплировали по одному часу, построили графики*

## Обучение

*Подготовим выборки и возьмем размер тестовой выборки 10% по условиям задачи.*

In [ ]:
train, test = train_test_split(taxi_week, shuffle=False, test_size=0.1)
train = train.dropna()
features_train = train.drop(['num_orders'], axis=1)
target_train = train['num_orders']
features_test = test.drop(['num_orders'], axis=1)
target_test = test['num_orders']

*Удалим пропуски в тренирвочной выборке методомм dropna().*

*Обучим модели. Возьмем модели линейной регрессии, слуайного леса и LGBMRegressor*

In [ ]:
splits = TimeSeriesSplit(gap=0, n_splits=5)

In [ ]:
model_lr = LinearRegression()

*Вычислим среднюю оценку качества каждой модели на кроссвалидации и выберем лучшую модель*

In [ ]:
scores = cross_val_score(model_lr, features_train, target_train, cv=splits, scoring='neg_root_mean_squared_error')
RMSE_1 = scores.sum()/len(scores)
print(f'Средняя оценка качества модели:, {-RMSE_1}')

*Следующая модель - случайный лес RandomForestRegressor*

In [ ]:
model_RFR = RandomForestRegressor(random_state=12345)

In [ ]:
param_grid = {
    'n_estimators': [50],
    'max_depth': [15]
}

In [ ]:
grid_search = GridSearchCV(model_RFR, param_grid, cv=splits, n_jobs=-1, scoring='neg_root_mean_squared_error')

In [ ]:
%%timeit
grid_search.fit(features_train, target_train)

In [ ]:
grid_best_score = grid_search.best_score_
grid_best_params = grid_search.best_params_


In [ ]:
print(f'Лучшие параметры: {grid_best_params}')
print(f'Лучша метрика: {grid_best_score*-1}')

In [ ]:
lgbm = LGBMRegressor(random_state=12345, verbose=-1)

In [ ]:
lgbm_parameters = {'n_estimators': [100, 500],
             'max_depth': [2, 6]}

*Найдем лучшие параметры c помощью RandomizedSearchCV*

In [ ]:
random_search = RandomizedSearchCV(estimator=lgbm, param_distributions=lgbm_parameters, scoring='neg_root_mean_squared_error', random_state=12345)

In [ ]:
%%timeit
random_search.fit(features_train, target_train)

In [ ]:
best_params = random_search.best_params_
best_score = random_search.best_score_

In [ ]:
print(f'Лучшие параметры: {best_params}')
print(f'Лучша метрика: {best_score*-1}')

*Лучшую среднюю оценку модели на кроссвалидации показала модель LGBMregressor, со значением RMSE 50,73  и c временем обучения 13.1 с. Выберем модель LGBMregressor и прогоним ее на тестовых данных*

## Тестирование

*Построим предсказания лучшей модели на тестовой выборке и посчитаем качество модели с помощью средней квадратичной ошибки RMSE*

In [ ]:
y_pred = random_search.predict(features_test)

In [ ]:
RMSE_test = mean_squared_error(target_test, y_pred)**0.5

In [ ]:
print(f'RMSE лучшей модели на тестовой выборке равна: {RMSE_test}')

**Вывод - Ошибка RMSE на тестовой выборке у лучшей модели равна 40.17, что меньше верхнего порога в 48 по условию задачи. Это значит, что наша лучшая модель работает хорошо на тестовых данных.**